## Wysyłanie żądania do Triton Inference Server w celu otrzymania rekomendacji


1. Należy uruchomić kontener `feedai-recsys-inference` w oparciu o definicję obrazu znajdującą się w pliku `docker\python_env\Dockerfile` za pomocą poleceń:

```shell
cd docker/python_env/
docker build -t feedai/merlin-tensorflow-inference:22.05 .
```
```shell
docker run -it --name feedai-recsys-inference --gpus all -p 8000:8000 -p 8001:8001 -p 8002:8002 -p 8888:8888 -v <path_to>/Building-and-deploying-multi-stage-RecSys:/Merlin/examples/Building-and-deploying-multi-stage-RecSys --ipc=host feedai/merlin-tensorflow-inference:22.05 /bin/bash
```

2. W uruchomionym kontenerze Należy uruchomić proces serwera JupyterLab oraz Triton Inference Server

```shell
jupyter-lab --allow-root --ip='0.0.0.0' --NotebookApp.token='lolo'
tritonserver --model-repository=/Merlin/examples/Building-and-deploying-multi-stage-RecSys/poc_ensemble/ --backend-config=tensorflow,version=2
```

3. Należy uruchomić bieżący notatnik na uruchomionym w kontenerze serwerze JupyterLab dostępnym pod `http://127.0.0.1`

##### Ustalenie schemy żądania do serwera oraz odpowiedzi

In [1]:
import numpy as np
from merlin.systems.triton.utils import send_triton_request
from merlin.core.dispatch import make_df

# create a request to be sent to TIS
request = make_df({"user_id": [1]})
request["user_id"] = request["user_id"].astype(np.int32)

# outputs = ensemble.graph.output_schema.column_names
outputs = ['ordered_ids']

In [2]:
request

,user_id
0,1


##### Wysłanie żądania do serwea, w odpowiedzi orzymujemy id rekomendowanych produktów

In [3]:
response = send_triton_request(request, outputs, endpoint="127.0.0.1:8001")
response

{'ordered_ids': array([[1189684],
        [1290349],
        [1908677],
        [ 955000],
        [2209519],
        [1240268],
        [1348758],
        [1034567],
        [1928709],
        [2816367]], dtype=int32)}